# Case A

This Method uses Cash-Karp Method !

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np 
import matplotlib.pyplot as plt
% matplotlib inline 

### Coupled Derivative to Intergrate

In [ ]:
def dydx(x,y):
    
    A = 1
    B = 0 
    C = 0
    D = 2/3
    
    y_derivs = np.zeros(2)
    
    y_derivs[0] = y[1]
    
    y_derivs[1] = -A*np.sin(y[0]) - B*y[1] + C*np.sin(D*x)
    
    return y_derivs

### 4th Order RK Method 

In [ ]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv) 
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    x_ipo = xi + h 
    
    y_temp2 = np.zeros(nv)
    y_temp3 = np.zeros(nv)
    y_temp4 = np.zeros(nv)
    y_temp5 = np.zeros(nv)
    y_temp6 = np.zeros(nv)
    
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:] 
    
    
    y_temp2[:] = yi[:] + (1/5)*k1[:]
    y_derivs = dydx(xi + (1/5)*h, y_temp2) 
    k2[:] = h*y_derivs[:] 
    
    
    y_temp3[:] = yi[:] + (3/40)*k1[:] + (9/40)*k2[:]
    y_derivs = dydx(xi + (3/10)*h, y_temp3)
    k3[:] = h*y_derivs[:]
    
    
    y_temp4[:] = yi[:] + (3/10)*k1[:] + -1*(9/10)*k2[:] + (6/5)*k3[:] 
    y_derivs = dydx(xi + (3/5)*h, y_temp4)
    k4[:] = h*y_derivs[:]
    
    
    y_temp5[:] = yi[:] + -1*(11/54)*k1[:] + (5/2)*k2[:] + -1*(70/27)*k3[:] + (35/27)*k4[:]
    y_derivs = dydx(xi + h, y_temp5)
    k5[:] = h*y_derivs[:]
    
    
    y_temp6[:] = yi[:] + (1631/55296)*k1[:] + (175/512)*k2[:] + (575/13824)*k3[:] + (44275/110592)*k4[:] + (253/4096)*k5[:]
    y_derivs = dydx(xi + (7/8)*h, y_temp6)
    k6[:] = h*y_derivs[:]
    
    
    yipo = yi + (37/378)*k1 + (0)*k2 + (250/621)*k3 + (125/594)*k4 + (0)*k5 + (512/1771)*k6
    
    return yipo 

## Adaptive Step Size for RK4

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    SAFETY = 0.9 
    H_NEW_FAC = 2.0 
    
    imax = 10000
    
    i = 0
    
    Delta = np.full(nv,2*tol) 
    
    h_step = h 
    
    while(Delta.max()/tol > 1.0):
        
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        Delta = np.fabs(y_2 - y_11)
        
        if(Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        if(i>=imax):
            print('Too many iterations in rk4_mv_ad()' )
            raise StopIteration ("Ending after i =",i)
            
        i += 1 
            
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
        
    return y_2, h_new, h_step

### Wrapper for RK4

In [ ]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    xi = a 
    yi = y_a.copy()
    
    h = 1.0e-4 * (b-a) 
    
    imax = 10000
    
    i = 0 
    
    nv = len(y_a) 
    
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    flag = 1
    
    while(flag): 
        
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        h = h_new
        
        if(xi+h_step>b):
            
            h = b-xi
            
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            flag = 0 
            
        xi += h_step 
        yi[:] = yi_new[:]
        
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y 
        y = y_new
        
        if(i>=imax): #missing = !!!!!
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        i += 1
        
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
        
        if(xi==b):
            flag = 0 
            
    return x,y # indented inside fxn

### Performing the Itergations

In [ ]:
a = 0.0 
b = 100.0

y_0 = np.zeros(2) 
y_0[0] = 0.0 
y_0[1] = 1.0 
nv = 2

tolerance = 1.0e-6

x,y = rk4_mv(dydx,a,b,y_0,tolerance)

In [ ]:
f, axarr = plt.subplots(3,1, figsize=(10,12))

axarr[0].plot(x, y[:,0], '-', label='y(x)')

axarr[0].set_xlabel('x')
axarr[0].set_ylabel('y, dy/dx')
axarr[0].set_title('Case A: Angle v. Time', color='C0')
axarr[0].axis()
#axarr[0].set_aspect(100)

axarr[1].plot(x, y[:,1], '-', color='orange', label='dydx(x)')
axarr[1].set_xlabel('x')
axarr[1].set_ylabel('dy^2/dx^2')
axarr[1].set_title('Case A: Ang Velocity v. Time', color='C1')

axarr[2].plot(y[:,1], y[:,0], '-', color='green', label='omega(theta)')

axarr[2].set_xlabel('theta')
axarr[2].set_ylabel('omega, dy/dx')
axarr[2].set_title('Case A: Ang Velocity vs. Angle', color='C2')

f.subplots_adjust(wspace = 1.0)
#fig = plt.figure(figsize = (6,6))

plt.subplots_adjust(hspace=0.5)

In [ ]:
sine = np.sin(x) 
cosine = np.cos(x)

y_error = (y[:,0] - sine)
dydx_error = (y[:,1] - cosine)

plt.plot(x, y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)

# Case B
This method uses RK4 method!

# Define our coupled derivatives to integrate

In [ ]:
def dydx(x, y): #A, B, C, D):
    
    # Set the derivatives
    
    # Our equation is d^2y/dx^2 = -A*sin(y)-B*z+C*sin(D*x)
    
    # So we can write
    #z = dydx
    #dzdx = -A*sin(y)-B*z+C*sin(D*x) = a
    
    # We will set a = y[0]
    # we will set z = y[1]
    
    # Declare an array
    y_derivs = np.zeros(2)
    
    # Set dydx = z
    y_derivs[0] = y[1]
    
    # Set dy^2dx^2 = a
    y_derivs[1] = -1*np.sin(y[0]) -(1/2)*y[1] + 0*np.sin((2/3)*x)
# Case B        (g/l)^0.5=1         q=1/2        Ad=0        OHMd=2/3
    
    # Here we have to return an array
    return y_derivs

### Define the 4th order RK method

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h):
    
    # Declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    # Define x at 1/2 step
    x_ipoh = xi +0.5*h
    
    # Define x at 1 step
    x_ipo = xi + h
    
    # Declare a temporary arrayy 
    y_temp = np.zeros (nv)
    
    # Get k1 values
    y_derivs = dydx(xi, yi)
    k1[:] = h*y_derivs[:]
    
    # Get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k2[:] = h*y_derivs[:]
    
    # Get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k3[:] = h*y_derivs[:]
    
    # Get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo, y_temp) #Why ipo and not ipoh?
    k4[:] = h*y_derivs[:]
    
    # Advance y by a step h
    yipo = yi + (k1 +2*k2 + 2*k3 + k4)/6.
    
    return yipo

### Define an adaptive step size driver for RK4

In [ ]:
def rk4_mv_ad(dydx, x_i, y_i, nv, h, tol):

    #define safetyscale
    SAFETY = .9
    H_NEW_FAC = 2.
    
    #Set a maximum number of iterations
    imax = 10000
    
    #Set an iteration varale
    i = 0
    
    #Create an error
    Delta = np.full(nv, 2*tol)
    
    #Remember the step
    h_step = h
    
    #Adjust the step
    while (Delta.max()/tol > 1.0):
        # Estimate our error by taking one step of size  h
        #vs. two steps of the size h/2
        y_2 = rk4_mv_core(dydx, x_i, y_i, nv, h_step)
        y_1 = rk4_mv_core(dydx, x_i, y_i, nv, 0.5*h_step)
        y_11 = rk4_mv_core(dydx, x_i+0.5*h_step, y_1, nv, 0.5*h_step)
        
        
        # Always try to make every step bigger than the other,
        #but they are always correlated, not just an array of
        #numbers together
        
        #Compute an error
        Delta = np.fabs(y_2 - y_11)
        
        # If the error is too large, take a smaller step
        if (Delta.max()/tol > 1.0):
            #our error is too large, degrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
         
        #Check iteration
        if(i >= imax):
            print ('Too may iteratctions in rk4_mv_ad()')
            raise StopIteration ("Ending after i =", i)
            
        #iterate
        i += 1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #Return the answer, a new stap, and the step we actually took
    return y_2, h_new, h_step

### Define a wrapper for RK4

In [ ]:
def rk4_mv(dydx, a, b, y_a, tol):
    
    #dydx is the derivative wrt x 
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set max number of iterations
    imax = 10000
    
    #Set an iteration variable
    i = 0
    
    #Set the number of coupled odes to the
    #size of y_a
    nv = len(y_a)
    
    #Set the initial conditions
    x = np.full(1,a)
    y = np.full((1, nv), y_a)
    
    #Set a flag
    flag = 1
    
    #Loop until we reach the right side
    while(flag):
        #caluclate y_i + 1
        yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
        
        #Update the step
        h = h_new
        
        #Prevent an overshoot
        if(xi+h_step > b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
            
            #break
            flag = 0
            
        #Update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np. append(x, xi)
        y_new = np.zeros((len(x), nv))
        y_new[0:len(x) - 1, :] = y
        y_new[-1, :] = yi[:]
        del y
        y = y_new
        
        #Prevent too many interactions
        if(i >= imax):
            print("MAximum iterations reachd.")
            raise StopIteration("Iteration number = ", i)
            
        #iterate
        i += 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
        
        #Break if new xi is == b
        if (xi == b):
            flag = 0
            
    #return the answer
    return x, y

### Perform the integration

In [ ]:
a = 0.0
b = 100.

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#Perform the integration
x, y = rk4_mv(dydx, a, b, y_0, tolerance)

### Plot the results

In [ ]:
f, axarr = plt.subplots(3,1, figsize=(10,12))

axarr[0].plot(x, y[:,0], '-', label='y(x)')

axarr[0].set_xlabel('x')
axarr[0].set_ylabel('y, dy/dx')
axarr[0].set_title('Case B: Angle v. Time', color='C0')
axarr[0].axis()
#axarr[0].set_aspect(100)

axarr[1].plot(x, y[:,1], '-', color='orange', label='dydx(x)')
axarr[1].set_xlabel('x')
axarr[1].set_ylabel('dy^2/dx^2')
axarr[1].set_title('Case B: Speed v. Time', color='C1')

axarr[2].plot(y[:,1], y[:,0], '-', color='green', label='omega(theta)')

axarr[2].set_xlabel('theta')
axarr[2].set_ylabel('omega, dy/dx')
axarr[2].set_title('Case B: Speed v. Angle', color='C2')

f.subplots_adjust(wspace = 1.0)
#fig = plt.figure(figsize = (6,6))

'''Aesthetic edits to the plots'''

#change plots border width
plt.rcParams['axes.linewidth'] = 1 #set the value globally
#change distance between plots
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)

### Plot the error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0] - sine)
dydx_error = (y[:,1] - cosine)

plt.plot(x, y_error, label='y(x) Error')
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)

# Case C

# Define our coupled derivatives to integrate

In [ ]:
def dydx(x, y): #A, B, C, D):
    
    # Set the derivatives
    
    # Our equation is d^2y/dx^2 = -A*sin(y)-B*z+C*sin(D*x)
    
    # So we can write
    #z = dydx
    #dzdx = -A*sin(y)-B*z+C*sin(D*x) = a
    
    # We will set a = y[0]
    # we will set z = y[1]
    
    # Declare an array
    y_derivs = np.zeros(2)
    
    # Set dydx = z
    y_derivs[0] = y[1]
    
    # Set dy^2dx^2 = a
    y_derivs[1] = -1*np.sin(y[0]) -(1/2)*y[1] + 0.5*np.sin((2/3)*x)
# Case C        (g/l)^0.5=1         q=1/2        Ad=0.5   OHMd=2/3
    
    # Here we have to return an array
    return y_derivs

### Define the 4th order RK method

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h):
    
    # Declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    # Define x at 1/2 step
    x_ipoh = xi +0.5*h
    
    # Define x at 1 step
    x_ipo = xi + h
    
    # Declare a temporary arrayy 
    y_temp = np.zeros (nv)
    
    # Get k1 values
    y_derivs = dydx(xi, yi)
    k1[:] = h*y_derivs[:]
    
    # Get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k2[:] = h*y_derivs[:]
    
    # Get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k3[:] = h*y_derivs[:]
    
    # Get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo, y_temp) #Why ipo and not ipoh?
    k4[:] = h*y_derivs[:]
    
    # Advance y by a step h
    yipo = yi + (k1 +2*k2 + 2*k3 + k4)/6.
    
    return yipo

### Define an adaptive step size driver for RK4

In [ ]:
def rk4_mv_ad(dydx, x_i, y_i, nv, h, tol):

    #define safetyscale
    SAFETY = .9
    H_NEW_FAC = 2.
    
    #Set a maximum number of iterations
    imax = 10000
    
    #Set an iteration varale
    i = 0
    
    #Create an error
    Delta = np.full(nv, 2*tol)
    
    #Remember the step
    h_step = h
    
    #Adjust the step
    while (Delta.max()/tol > 1.0):
        # Estimate our error by taking one step of size  h
        #vs. two steps of the size h/2
        y_2 = rk4_mv_core(dydx, x_i, y_i, nv, h_step)
        y_1 = rk4_mv_core(dydx, x_i, y_i, nv, 0.5*h_step)
        y_11 = rk4_mv_core(dydx, x_i+0.5*h_step, y_1, nv, 0.5*h_step)
        
        
        # Always try to make every step bigger than the other,
        #but they are always correlated, not just an array of
        #numbers together
        
        #Compute an error
        Delta = np.fabs(y_2 - y_11)
        
        # If the error is too large, take a smaller step
        if (Delta.max()/tol > 1.0):
            #our error is too large, degrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
         
        #Check iteration
        if(i >= imax):
            print ('Too may iteratctions in rk4_mv_ad()')
            raise StopIteration ("Ending after i =", i)
            
        #iterate
        i += 1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #Return the answer, a new stap, and the step we actually took
    return y_2, h_new, h_step

### Define a wrapper for RK4

In [ ]:
def rk4_mv(dydx, a, b, y_a, tol):
    
    #dydx is the derivative wrt x 
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set max number of iterations
    imax = 10000
    
    #Set an iteration variable
    i = 0
    
    #Set the number of coupled odes to the
    #size of y_a
    nv = len(y_a)
    
    #Set the initial conditions
    x = np.full(1,a)
    y = np.full((1, nv), y_a)
    
    #Set a flag
    flag = 1
    
    #Loop until we reach the right side
    while(flag):
        #caluclate y_i + 1
        yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
        
        #Update the step
        h = h_new
        
        #Prevent an overshoot
        if(xi+h_step > b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
            
            #break
            flag = 0
            
        #Update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np. append(x, xi)
        y_new = np.zeros((len(x), nv))
        y_new[0:len(x) - 1, :] = y
        y_new[-1, :] = yi[:]
        del y
        y = y_new
        
        #Prevent too many interactions
        if(i >= imax):
            print("MAximum iterations reachd.")
            raise StopIteration("Iteration number = ", i)
            
        #iterate
        i += 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
        
        #Break if new xi is == b
        if (xi == b):
            flag = 0
            
    #return the answer
    return x, y

### Perform the integration

In [ ]:
a = 0.0
b = 100.

#A
#B
#C
#D


y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#Perform the integration
x, y = rk4_mv(dydx, a, b, y_0, tolerance)

### Plot the results

In [ ]:
f, axarr = plt.subplots(3,1, figsize=(10,12))

axarr[0].plot(x, y[:,0], '-', label='y(x)')

axarr[0].set_xlabel('x')
axarr[0].set_ylabel('y, dy/dx')
axarr[0].set_title('Case C: angle v. time', color='C0')
axarr[0].axis()
#axarr[0].set_aspect(100)

axarr[1].plot(x, y[:,1], '-', color='orange', label='dydx(x)')
axarr[1].set_xlabel('x')
axarr[1].set_ylabel('dy^2/dx^2')
axarr[1].set_title('Case C: speed v. time', color='C1')

axarr[2].plot(y[:,1], y[:,0], '-', color='green', label='omega(theta)')

axarr[2].set_xlabel('theta')
axarr[2].set_ylabel('omega, dy/dx')
axarr[2].set_title('Case C: speed v. angle', color='C2')

f.subplots_adjust(wspace = 1.0)
#fig = plt.figure(figsize = (6,6))

'''Aesthetic edits to the plots'''

#change plots border width
plt.rcParams['axes.linewidth'] = 1 #set the value globally
#change distance between plots
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)

### Plot the error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0] - sine)
dydx_error = (y[:,1] - cosine)

plt.plot(x, y_error, label='y(x) Error')
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)

# Case D
This Method uses Cash-Karp Method !

### Coupled Derivative to Intergrate

In [ ]:
def dydx(x,y):
    
    A = 1
    B = 0.5
    C = 1.46
    D = 2/3
    
    y_derivs = np.zeros(2)
    
    y_derivs[0] = y[1]
    
    y_derivs[1] = -A*np.sin(y[0]) - B*y[1] + C*np.sin(D*x)
    
    return y_derivs

### 4th Order RK Method 

In [ ]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv) 
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    x_ipo = xi + h 
    
    y_temp2 = np.zeros(nv)
    y_temp3 = np.zeros(nv)
    y_temp4 = np.zeros(nv)
    y_temp5 = np.zeros(nv)
    y_temp6 = np.zeros(nv)
    
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:] 
    
    
    y_temp2[:] = yi[:] + (1/5)*k1[:]
    y_derivs = dydx(xi + (1/5)*h, y_temp2) 
    k2[:] = h*y_derivs[:] 
    
    
    y_temp3[:] = yi[:] + (3/40)*k1[:] + (9/40)*k2[:]
    y_derivs = dydx(xi + (3/10)*h, y_temp3)
    k3[:] = h*y_derivs[:]
    
    
    y_temp4[:] = yi[:] + (3/10)*k1[:] + -1*(9/10)*k2[:] + (6/5)*k3[:] 
    y_derivs = dydx(xi + (3/5)*h, y_temp4)
    k4[:] = h*y_derivs[:]
    
    
    y_temp5[:] = yi[:] + -1*(11/54)*k1[:] + (5/2)*k2[:] + -1*(70/27)*k3[:] + (35/27)*k4[:]
    y_derivs = dydx(xi + h, y_temp5)
    k5[:] = h*y_derivs[:]
    
    
    y_temp6[:] = yi[:] + (1631/55296)*k1[:] + (175/512)*k2[:] + (575/13824)*k3[:] + (44275/110592)*k4[:] + (253/4096)*k5[:]
    y_derivs = dydx(xi + (7/8)*h, y_temp6)
    k6[:] = h*y_derivs[:]
    
    
    yipo = yi + (37/378)*k1 + (0)*k2 + (250/621)*k3 + (125/594)*k4 + (0)*k5 + (512/1771)*k6
    
    return yipo 

### Adaptive Step Size for RK4

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    SAFETY = 0.9 
    H_NEW_FAC = 2.0 
    
    imax = 10000
    
    i = 0
    
    Delta = np.full(nv,2*tol) 
    
    h_step = h 
    
    while(Delta.max()/tol > 1.0):
        
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        Delta = np.fabs(y_2 - y_11)
        
        if(Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        if(i>=imax):
            print('Too many iterations in rk4_mv_ad()' )
            raise StopIteration ("Ending after i =",i)
            
        i += 1 
            
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
        
    return y_2, h_new, h_step

### Wrapper for RK4

In [ ]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    xi = a 
    yi = y_a.copy()
    
    h = 1.0e-4 * (b-a) 
    
    imax = 10000
    
    i = 0 
    
    nv = len(y_a) 
    
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    flag = 1
    
    while(flag): 
        
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        h = h_new
        
        if(xi+h_step>b):
            
            h = b-xi
            
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            flag = 0 
            
        xi += h_step 
        yi[:] = yi_new[:]
        
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y 
        y = y_new
        
        if(i>=imax): #missing = !!!!!
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        i += 1
        
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
        
        if(xi==b):
            flag = 0 
            
    return x,y # indented inside fxn
    

### Performing the Itergations 

In [ ]:
a = 0.0 
b = 100.0

y_0 = np.zeros(2) 
y_0[0] = 0.0 
y_0[1] = 1.0 
nv = 2

tolerance = 1.0e-6

x,y = rk4_mv(dydx,a,b,y_0,tolerance)

### Plot the results

In [ ]:
f, axarr = plt.subplots(3,1, figsize=(10,12))

axarr[0].plot(x, y[:,0], '-', label='y(x)')

axarr[0].set_xlabel('x')
axarr[0].set_ylabel('y, dy/dx')
axarr[0].set_title('Case D: angle v. time', color='C0')
axarr[0].axis()
#axarr[0].set_aspect(100)

axarr[1].plot(x, y[:,1], '-', color='orange', label='dydx(x)')
axarr[1].set_xlabel('x')
axarr[1].set_ylabel('dy^2/dx^2')
axarr[1].set_title('Case D: Ang Velocity v. Time', color='C1')

axarr[2].plot(y[:,1], y[:,0], '-', color='green', label='omega(theta)')

axarr[2].set_xlabel('theta')
axarr[2].set_ylabel('omega, dy/dx')
axarr[2].set_title('Case D: Ang Velocity vs. Angle', color='C2')

f.subplots_adjust(wspace = 1.0)
#fig = plt.figure(figsize = (6,6))

plt.subplots_adjust(hspace=0.5)

### Plot the error

In [ ]:
sine = np.sin(x) 
cosine = np.cos(x)

y_error = (y[:,0] - sine)
dydx_error = (y[:,1] - cosine)

plt.plot(x, y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)